In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [95]:
train=pd.read_csv('f:/ym4trainsparse.csv',usecols=[1,2,3])
test=pd.read_csv('f:/ym4testsparse.csv',usecols=[1,2,3])

In [96]:
no_movies = 1001
no_users = 15401

In [97]:
import time

In [98]:
rated= [set() for i in range(no_movies) ]
dictionary={}

In [99]:
deviation=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)
cardinality=np.zeros(no_movies*no_movies).reshape(no_movies,no_movies)

In [100]:
for i, row in enumerate(train.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.movieId)
    rate=row.rating
    
    rated[mov].add(usr)
    dictionary[(usr,mov)]=rate

In [101]:
for i in range(no_movies):
    t1=time.time()
    
    for j in range(i+1,no_movies):
        tmprate = list(rated[i].intersection(rated[j]))
        
        uoutput=0
        c=0
        for uid in tmprate:
            uoutput=uoutput+(dictionary[(uid,j)]-dictionary[(uid,i)])
            c=c+1
            
        deviation[i][j]=uoutput
        deviation[j][i]=-uoutput
        cardinality[i][j]=c
        cardinality[j][i]=c

In [102]:
test.describe()

,userId,movieId,rating
count,10551.000000,10551.000000,10551.000000
mean,10053.692446,496.290115,2.960478
std,3182.642263,304.551041,1.594556
min,2223.000000,1.000000,1.000000
25%,7487.500000,229.500000,1.000000
50%,10125.000000,492.000000,3.000000
75%,12762.500000,780.000000,5.000000
max,15400.000000,1000.000000,5.000000


In [103]:
number=test['userId'].count()

In [104]:
import math

In [105]:
rmse_normal2=0
mae_normal2=0
err_gim=0
err_gpim=0
c_gim=0
c_gpim=0
tp=0
fp=0
tn=0
fn=0
for i, row in enumerate(test.itertuples(), 1):
    usr=int(row.userId)
    mov=int(row.movieId)
    actual=row.rating
    pred_normal2=0
    pred_weighted2=0
    sum_weighted2=0
    list_of_all_movies_rate_by_the_current_user=train[train['userId']==usr]
    length=list_of_all_movies_rate_by_the_current_user.shape[0]
    cnt=0
    for j,r2 in enumerate(list_of_all_movies_rate_by_the_current_user.itertuples(),1):
        curmov=r2.movieId
        currate=r2.rating
        if cardinality[curmov][mov]!=0:
            pred_normal2+=(currate+(deviation[curmov][mov]/ cardinality[curmov][mov]))
            cnt+=1
    if cnt!=0:
        predicted_normal2=(pred_normal2/cnt)
    else:
        predicted_normal2=0
    if predicted_normal2>5:
        predicted_normal2=5
    if predicted_normal2 <0:
        predicted_normal2=0
    
    if actual >=3:
        err_gim+=math.fabs(actual-predicted_normal2)
        c_gim+=1
    if predicted_normal2>=3:
        err_gpim+=math.fabs(actual-predicted_normal2)
        c_gpim+=1
    if actual>=3 and predicted_normal2>=3:
        tp+=1
    elif actual<3 and predicted_normal2>=3:
        fp+=1
    elif actual>=3 and predicted_normal2<3:
        fn+=1
    else:
        tn+=1
    rmse_normal2+=((actual-predicted_normal2)**2)
    mae_normal2+=abs(actual-predicted_normal2)
    

In [106]:
print(mae_normal2/number)

1.37586296232


In [107]:
print(math.sqrt(rmse_normal2/number))

1.8328701348911534


In [108]:
err_gim/c_gim

1.4116273445592038

In [109]:
err_gpim/c_gpim

1.1654578209149706

In [110]:
tp/(tp+fp)

0.7392339749950387

In [111]:
tp/(tp+fn)

0.613471673254282